## Import Libraries and set Configurations 

In [4]:
import sempy.fabric as fabric
import requests
import json
import time
import base64

# Constants (replace with actual values)
WORKSPACE_1_ID = fabric.get_workspace_id()  #Dev Workspace Id
WORKSPACE_2_ID = "e106bd6d-a2c9-433c-9cad-405d5522d245"
WORKSPACE_2_NAME = "J_Workspace_Test"
SEMANTIC_MODEL_ID = "69129b16-5add-4ae8-b655-ee298bb66465"
REPORT_ID="7ac111de-7e45-4eaf-bfac-ff1ed799ff44"
SEMANTIC_MODEL_NAME = "Employee Dept Semantic Model"
SEMANTIC_MODEL_DESCRIPTION = "Semantic Model for Employee model"

StatementMeta(, f72334c3-f5f3-444b-8dfe-7acf39da9396, 8, Finished, Available, Finished)

## <mark>Semantic Model Deployment from Dev to Test and Prod Workspace</mark>

1. Get the Definition of Existing Dev Semantic Model by Using [Get Definition](https://learn.microsoft.com/en-us/rest/api/fabric/semanticmodel/items/get-semantic-model-definition?tabs=HTTP) API 
2. Modify the Model Definition Name and Descriptions (if required) by using Higher Environment(Test or Prod) Workspace Id
3. Use [Create Semantic Model](https://learn.microsoft.com/en-us/rest/api/fabric/semanticmodel/items/create-semantic-model?tabs=HTTP) API With Model definition as request payload
4. Use [Long Running Operation API](https://learn.microsoft.com/en-us/rest/api/fabric/core/long-running-operations/get-operation-state?tabs=HTTP)s to get the status and result
5. Get the Newly created Semantic Model Id for higher environment (Test or Prod)

In [5]:
# Get authentication token
fabric_api_scope = "https://api.fabric.microsoft.com/"
token = mssparkutils.credentials.getToken(fabric_api_scope)
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Step 1: Get semantic model definition from Workspace 1
print(f"Get the Definition of Existing Semantic Model : {SEMANTIC_MODEL_ID} from Workspace 1")
get_definition_url = f"{fabric_api_scope}v1/workspaces/{WORKSPACE_1_ID}/semanticModels/{SEMANTIC_MODEL_ID}/getDefinition"
response = requests.post(get_definition_url, headers=headers)
operation_id = response.headers.get("x-ms-operation-id")
print(f"getDefinition Operation ID: {operation_id}")

# Step 2: Poll for operation completion
print("*************Polling for Get Definition operation completion*************")
operation_status_url = f"{fabric_api_scope}v1/operations/{operation_id}"
while True:
    status_response = requests.get(operation_status_url, headers=headers)
    status = status_response.json().get("status")
    print(f"getDefinition Operation status: {status} .. ")
    if status == "Succeeded":
        break
    elif status == "Failed":
        raise Exception("getDefinition Operation failed")
    time.sleep(5)

# Step 3: Retrieve operation result
result_url = f"{operation_status_url}/result"
result_response = requests.get(result_url, headers=headers)
semantic_model_definition = result_response.json()

# Step 4: Update model metadata
semantic_model_definition["displayName"] = SEMANTIC_MODEL_NAME
semantic_model_definition["description"] = SEMANTIC_MODEL_DESCRIPTION

# Optional: Save definition to file
with open("semantic_model_definition.json", "w") as file:
    json.dump(semantic_model_definition, file, indent=2)

# Step 5: Create semantic model in Workspace 2
print("Create Semantic Model...")
create_model_url = f"{fabric_api_scope}v1/workspaces/{WORKSPACE_2_ID}/semanticModels"
create_response = requests.post(create_model_url, headers=headers, json=semantic_model_definition)
create_operation_id = create_response.headers.get("x-ms-operation-id")
print(f"Create semanticModels Operation ID: {create_operation_id}")

# Step 6: Poll for creation completion
print("*************Polling for Create semanticModels operation completion*************")
create_status_url = f"{fabric_api_scope}v1/operations/{create_operation_id}"
while True:
    create_status_response = requests.get(create_status_url, headers=headers)
    create_status = create_status_response.json().get("status")
    print(f"Create semanticModels Operation status: {create_status}")
    if create_status == "Succeeded":
        break
    elif create_status == "Failed":
        print("Create semanticModels operation failed:", json.dumps(create_status_response.json(), indent=2))
        break
    time.sleep(10)

# Step 7: Retrieve new semantic model ID
create_result_url = f"{create_status_url}/result"
create_result_response = requests.get(create_result_url, headers=headers)
create_result = create_result_response.json()
if create_status == "Succeeded":
    new_model_id = create_result.get("id")
    print(f"New Semantic Model ID in Workspace 2: {new_model_id}")
else:
    print("Error details from result endpoint:", json.dumps(create_result, indent=2))


StatementMeta(, f72334c3-f5f3-444b-8dfe-7acf39da9396, 9, Finished, Available, Finished)

Get the Definition of Existing Semantic Model : 69129b16-5add-4ae8-b655-ee298bb66465 from Workspace 1
getDefinition Operation ID: c7f403c8-fe2f-4e83-990d-aafd65f765d6
*************Polling for Get Definition operation completion*************
getDefinition Operation status: Running .. 
getDefinition Operation status: Succeeded .. 
Create Semantic Model...
Create semanticModels Operation ID: 8188b02e-214b-41aa-a8af-14f353890f9f
*************Polling for Create semanticModels operation completion*************
Create semanticModels Operation status: Running
Create semanticModels Operation status: Running
Create semanticModels Operation status: Succeeded
New Semantic Model ID in Workspace 2: c8aac368-cfe2-478f-919b-8ca1e3d9fcb3


# <mark>Power BI Report deployment from Dev to Test / Prod</mark>
1. Get the Definition of Existing Dev Power BI Report by Using [Get Definition](https://learn.microsoft.com/en-us/rest/api/fabric/report/items/get-report-definition?tabs=HTTP) API 
2. Modify the Report Name and Descriptions (if required) by using Higher Environment(Test or Prod) Workspace Id
3. Use [Create Report](https://learn.microsoft.com/en-us/rest/api/fabric/report/items/create-report?tabs=HTTP) API With updated definition as request payload
4. Use [Long Running Operation API](https://learn.microsoft.com/en-us/rest/api/fabric/core/long-running-operations/get-operation-state?tabs=HTTP)s to get the status and result

In [6]:
# Get authentication token
fabric_api_scope = "https://api.fabric.microsoft.com/"
token = mssparkutils.credentials.getToken(fabric_api_scope)
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Step 1: Get definition of report from workspace-1
get_report_def_url = f"https://api.fabric.microsoft.com/v1/workspaces/{WORKSPACE_1_ID}/reports/{REPORT_ID}/getDefinition"
report_response = requests.post(get_report_def_url, headers=headers)
report_response.raise_for_status()
report_operation_id = report_response.headers.get("x-ms-operation-id")
print(f"Report Operation ID: {report_operation_id}")

# Step 2: Wait for long operation to complete
report_status_url = f"https://api.fabric.microsoft.com/v1/operations/{report_operation_id}"
while True:
    status_response = requests.get(report_status_url, headers=headers)
    status_response.raise_for_status()
    status = status_response.json()["status"]
    print(f"Report Operation status: {status}")
    if status == "Succeeded":
        break
    elif status == "Failed":
        raise Exception("Report operation failed")
    time.sleep(5)

# Step 3: Get operation result
report_result_url = f"https://api.fabric.microsoft.com/v1/operations/{report_operation_id}/result"
report_result_response = requests.get(report_result_url, headers=headers)
report_result_response.raise_for_status()
report_definition = report_result_response.json()

# Step 4: Add displayName and description to the definition
report_definition["displayName"] = "Employee Department Report"
report_definition["description"] = "Employee Department Report Details"


# Update the semantic model reference in definition.pbir . this contains the connection string. Update as per each env
definition_pbir = {
    "$schema": "https://developer.microsoft.com/json-schemas/fabric/item/report/definitionProperties/2.0.0/schema.json",
    "version": "4.0",
    "datasetReference": {
        "byConnection": {
            "connectionString": f"Data Source=powerbi://api.powerbi.com/v1.0/myorg/{WORKSPACE_2_NAME};initial catalog={SEMANTIC_MODEL_NAME};integrated security=ClaimsToken;semanticmodelid={new_model_id}"
        }
    }
}


# Encode the new definition.pbir content in InlineBase64 format
definition_encoded = base64.b64encode(json.dumps(definition_pbir).encode("utf-8")).decode("utf-8")

# Replace the payload for the part with path "definition.pbir"
for part in report_definition.get("definition", {}).get("parts", []):
    if part.get("path") == "definition.pbir":
        part["payload"] = definition_encoded
        part["payloadType"] = "InlineBase64"
        break

# Save the updated report definition to a new file
with open("updated_report_definition.json", "w") as f:
    json.dump(report_definition, f, indent=2)

print("Updated the payload for 'definition.pbir' and saved to 'updated_report_definition.json'.")



# Step 5: Create report in workspace 2
create_report_url = f"https://api.fabric.microsoft.com/v1/workspaces/{WORKSPACE_2_ID}/reports"
create_report_response = requests.post(create_report_url, headers=headers, json=report_definition)
create_report_response.raise_for_status()
create_report_operation_id = create_report_response.headers.get("x-ms-operation-id")
print(f"Create Report Operation ID: {create_report_operation_id}")

# Step 6: Wait for creation operation to complete
create_report_status_url = f"https://api.fabric.microsoft.com/v1/operations/{create_report_operation_id}"
while True:
    create_status_response = requests.get(create_report_status_url, headers=headers)
    create_status_response.raise_for_status()
    create_status = create_status_response.json()["status"]
    print(f"Create Report Operation status: {create_status}")
    if create_status == "Succeeded":
        break
    elif create_status == "Failed":
        raise Exception("Create report operation failed")
    time.sleep(5)

# Step 7: Get result and new report ID
create_report_result_url = f"https://api.fabric.microsoft.com/v1/operations/{create_report_operation_id}/result"
create_report_result_response = requests.get(create_report_result_url, headers=headers)
new_report_id = create_report_result_response.json()["id"]
print(f"New Report ID in Workspace-2: {new_report_id}")


StatementMeta(, f72334c3-f5f3-444b-8dfe-7acf39da9396, 10, Finished, Available, Finished)

Report Operation ID: d16d5507-282f-4f72-889a-139b9cda8b4f
Report Operation status: Succeeded
Updated the payload for 'definition.pbir' and saved to 'updated_report_definition.json'.
Create Report Operation ID: b09fa183-1a47-4047-95a0-e49185ffbed2
Create Report Operation status: Running
Create Report Operation status: Succeeded
New Report ID in Workspace-2: d3865f91-c9b9-4b3c-999c-abaf8e0a1428


In [ ]:
# Report creation